# MIE 1624: Final Project

#### Group 22 Members

Aaron Hao Tan, Gurtej Bhasin, Elise Emma, Jacob Bulir, Alex Kwan

## Introduction 

< -- Insert brief intro to project -- >

## Table of Content 

#### 0. Preliminaries
#### 1. Data Cleaning
    - 1.1 Claim Cleaning
    - 1.2 Related Articles Cleaning
        - 1.2.1 Extract Sentences
    - 1.3 Removing Stopwords and Stemming
        - 1.3.1 Claims
        - 1.3.2 Related Articles
    - 1.4 Date Cleaning
    - 1.5 Claimant Cleaning
    - 1.6 Finalize Cleaned Data
        - 1.6.1 Finalize Train Data
        - 1.6.2 Finalzie Related Articles Data
#### 2. Exploratory Data Analysis
    - 2.1 Date Features
    - 2.2 Claimant Features
    - 2.3 Doc2Vec Features
        - 2.3.1 Preprocess the Data
        - 2.3.2 Prepare for Training
        - 2.3.3 Doc2Vec Training
        - 2.3.4 Find Similarities
            - 2.3.4.1 Finding Top 5 Sentences per Related Articles per Claim
            - 2.3.4.2 Finding Top 5 Sentences Amongst all Related Articles per Claim
        - 2.3.5 Feature Encoding
            - 2.3.5.1 Feature Encode the Claims
            - 2.3.5.2 Feature Encode the Top 5 Related Article Sentences for Each Claim
            - 2.3.5.3 Feature Encode the Similarity Scores
        - 2.3.6 Combine All Features (Claim Vector, Top 5 Related Sentence Vector, Similarity Scores)
    - 2.4 Sentiment Features
    - 2.5 Exploration of Features
#### 3. Feature Selection
#### 4. Model Implementation
#### 5. Model Tuning
#### 6. Results

## Preliminaries

In [70]:
#----------------------------------------------
# Import Libraries
#----------------------------------------------
import pandas as pd
import numpy as np
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
import re
from datetime import datetime
import os
import math
from IPython.display import clear_output, display
import time
import warnings
warnings.filterwarnings('ignore')
import string
import heapq
from heapq import nlargest
from operator import itemgetter

# The following line is needed to show plots inline in notebooks
%matplotlib inline 

## 1. Data Cleaning 

In this section, the train.json file as well as the related articles are cleaned. The output of this section are two data frames that hold the cleaned information. The following shows the subsections:

- 1.1 Claim Cleaning
- 1.2 Related Articles Cleaning
    - 1.2.1 Extract Sentences
- 1.3 Removing Stopwords and Stemming
    - 1.3.1 Claims
    - 1.3.2 Related Articles
- 1.4 Date Cleaning
- 1.5 Claimant Cleaning
- 1.6 Finalize Cleaned Data
    - 1.6.1 Finalize Train Data
    - 1.6.2 Finalzie Related Articles Data

### 1.1 Claim Cleaning

Read the "train.json" file and perform some basic cleaning on the claims. This includes the following key points:

    - Lowercase all claims
    - Remove "\n" new lines
    - Remove URL links
    - Remove unicode hex
    - Remove punctuations
    - Remove misc. items such as "-", "'" ... etc.
    - Remove any extra spaces

In [2]:
# read train.json file for training data
data = pd.read_json("train.json")
data.shape

(15555, 6)

In [3]:
%%time

# CLEAN CLAIM DATA

# create a new dataframe of just claims
cleaned_claim = data.claim.to_frame()

# convert all string values to lower case
cleaned_claim = cleaned_claim.apply(lambda x: x.str.lower())

# replace new line with space
cleaned_claim = cleaned_claim.replace('\n', ' ', regex=True)

# get rid of all links
cleaned_claim = cleaned_claim.claim.replace(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', regex = True).to_frame()

# get rid of unicode hex
cleaned_claim = cleaned_claim.claim.replace({r'[^\x00-\x7F]+':''}, regex=True).to_frame()

# remove punctuation
cleaned_claim = cleaned_claim.claim.str.replace('[{}]'.format(string.punctuation), '').to_frame()

# remove misc items
cleaned_claim = cleaned_claim.replace(' — ', ' ', regex=True)
cleaned_claim = cleaned_claim.replace('-', ' ', regex=True)
cleaned_claim = cleaned_claim.replace('’', '', regex=True)
cleaned_claim = cleaned_claim.replace('‘', '', regex=True)
cleaned_claim = cleaned_claim.replace('”', '', regex=True)
cleaned_claim = cleaned_claim.replace('“', '', regex=True)

# replace consecutive spaces with just one space
cleaned_claim = cleaned_claim.replace('\s+', ' ', regex=True)

CPU times: user 445 ms, sys: 2.7 ms, total: 448 ms
Wall time: 447 ms


For ease of manipulation, the related articles information are converted in to numpy arrays with the following code segment.

In [4]:
# convert related_articles list to a row of array

article_array = []
for i in range(data.shape[0]):
    array = np.asarray(data['related_articles'].loc[i])
    article_array.append(array)
    
data['article_array'] = article_array

The cleaned claims, labels and the related articles in array format are shown below.

In [5]:
# concatenate cleaned_claims with label and article_array
cleaned_claim = pd.concat([cleaned_claim, data.label, data.article_array], axis=1)

# cleaned_claim now holds the claims that are cleaned, the label, and the article array
cleaned_claim.head()

,claim,label,article_array
0,a line from george orwells novel 1984 predicts...,0,"[122094, 122580, 130685, 134765]"
1,maine legislature candidate leslie gibson insu...,2,"[106868, 127320, 128060]"
2,a 17yearold girl named alyssa carson is being ...,1,"[132130, 132132, 149722]"
3,in 1988 author roald dahl penned an open lette...,2,"[123254, 123418, 127464]"
4,when it comes to fighting terrorism another th...,2,"[41099, 89899, 72543, 82644, 95344, 88361]"


### 1.2 Related Articles Cleaning

Similar to the procedures used to clean the claims, we clean the related articles in this section. To begin, we first create a dataframe that holds every related article. The index in this case are the article ID's.

In [6]:
%%time

# set paths
cur_path = os.path.dirname(os.path.abspath("Final_Project.ipynb"))
articles_dir = cur_path + '/train_articles/'

# create a dictionary of article ID and content
article_dict = {}
for filename in os.listdir(articles_dir):
    filenumber = filename.replace('.txt', '')
    file_open = open(articles_dir + filename, "r")
    text = file_open.read()
    article_dict[filenumber] = text
    
# use the dictionary created to create a dataframe of articles
articles  = pd.DataFrame.from_dict(article_dict, orient='index')
articles.columns = ['Article']

# a dataframe that holds all the articles
articles.head()

CPU times: user 1.44 s, sys: 459 ms, total: 1.9 s
Wall time: 1.91 s


,Article
125923,FACT CHECK: The Five Pillars of Curriculum\nFA...
57270,"The State Department Spent $52,701 on the Curt..."
144732,US show of force sends Russia a message in Bla...
28766,Mexico missing students: Protesters clash with...
85305,The U.S. Refugee Resettlement Program: A Prime...


With the related articles dataframe, the following performs the basic cleaning that was also applied to the claims.

In [7]:
%%time

# CLEAN ARTICLE DATA - ~5 minutes to run

# convert all string values to lower case
articles_cleaned = articles.apply(lambda x: x.str.lower())

# replace new line with space
articles_cleaned = articles_cleaned.replace('\n', ' ', regex=True)

# get rid of all links
articles_cleaned = articles_cleaned.Article.replace(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', regex = True).to_frame()

# get rid of unicode hex
articles_cleaned = articles_cleaned.Article.replace({r'[^\x00-\x7F]+':''}, regex=True).to_frame()

# remove misc items
articles_cleaned = articles_cleaned.replace(' — ', ' ', regex=True)
articles_cleaned = articles_cleaned.replace('-', '', regex=True)
articles_cleaned = articles_cleaned.replace('’', '', regex=True)
articles_cleaned = articles_cleaned.replace('‘', '', regex=True)
articles_cleaned = articles_cleaned.replace('”', '', regex=True)
articles_cleaned = articles_cleaned.replace('“', '', regex=True)

# replace consecutive spaces with just one space
articles_cleaned = articles_cleaned.replace('\s+', ' ', regex=True)

CPU times: user 1min 54s, sys: 653 ms, total: 1min 55s
Wall time: 1min 55s


### 1.2.1 Extract Sentences from Related Articles

In this section, we perform sentence tokenization on all the related articles to extract individual sentences from each article. Puntuations are then removed from the extracted sentences.

In [8]:
%%time

# split each article in to sentences by "sent_tokenize"
from nltk import sent_tokenize
article_sentences = []
for i in range(articles_cleaned.shape[0]):
    sentence = articles_cleaned.Article[articles_cleaned.index[i]]
    tok_sen = sent_tokenize(sentence)
    article_sentences.append(tok_sen)

CPU times: user 2min 2s, sys: 552 ms, total: 2min 2s
Wall time: 2min 3s


In [9]:
# remove punctuation from cleaned articles - had to do this after the sentences are separated
articles_cleaned = articles_cleaned.Article.str.replace('[{}]'.format(string.punctuation), '').to_frame()

In [10]:
%%time

# remove punctuation from sentences
cleaned_article_sent = []
for i in range(articles_cleaned.shape[0]):
    cleaned_sent = [''.join(c for c in s if c not in string.punctuation) for s in article_sentences[i]]
    # cleaned_article_sent is a list of lists
    # the sentences from each article is grouped in to a list, and then grouped together based on article
    cleaned_article_sent.append(cleaned_sent)

CPU times: user 50.4 s, sys: 388 ms, total: 50.8 s
Wall time: 50.8 s


The following shows the article sentences dataframe, where each cell holds a list of sentences, indexed by the related article ID.

In [11]:
# convert the cleaned_Article_sent in to a dataframe that's indexed by article ID
article_sentences = pd.Series(cleaned_article_sent).to_frame()
article_sentences.columns = ['Sentence']
article_sentences.index = articles_cleaned.index

# article sentences in a dataframe
article_sentences.head()

,Sentence
125923,[fact check the five pillars of curriculum fac...
57270,[the state department spent 52701 on the curta...
144732,[us show of force sends russia a message in bl...
28766,[mexico missing students protesters clash with...
85305,[the us refugee resettlement program a primer ...


### 1.3 Remove Stopwords and Stemming

Now that the basic cleaning and sentence extraction are completed, the next step is to remove stopwords and perform stemming on the remaining words. The purpose is to reduce the vocabulary space by keeping only the words that are meaningful.The following imports the necessary libraries to conduct these tasks.

In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
# the first time running - you may need to uncomment the bottom two lines to download the necessary packages
# nltk.download('stopwords')
# nltk.download('punkt')

### 1.3.1 Claims

This section removes the stopwords and stems the claims.

In [13]:
# create a list of claims
claim_list=[]
for i in range(cleaned_claim.shape[0]):
    claim_entry = cleaned_claim.claim.loc[i]
    claim_list.append(claim_entry)

In [14]:
%%time
# tokenize every claim in the claim list generated from above
# the result is a list of tokenized claims: tokenized_claims
tokenized_claims = []
stemmed_claims = []
stemmed_sw_claims = []
for i in range(cleaned_claim.shape[0]):

    #--------------------------------------------------------------
    # stemming
    word_tokens = word_tokenize(claim_list[i])
    stemmed_tok_claims = []
    for w in word_tokens:
        stemmed_tok_claims.append(ps.stem(w))
    stemmed_string = ' '.join(stemmed_tok_claims)
    # stemmed_claims is a list of stemmed strings
    stemmed_claims.append(stemmed_string)
    
    #--------------------------------------------------------------
    # remove stop words
    stemmed_sw_string = []
    word_tokens = word_tokenize(stemmed_claims[i])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    stemmed_sw_string = ' '.join(filtered_sentence)
    # stemmed_sw_claims is a list of stemmed strings without stopwords
    stemmed_sw_claims.append(stemmed_sw_string)    
        
    #--------------------------------------------------------------    
    # tokenize
    tokenized_ = word_tokenize(stemmed_sw_claims[i])
    tokenized_claims.append(tokenized_)
    
    # print progress
    progress = round((i/cleaned_claim.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

progress: 99.99%
CPU times: user 34.6 s, sys: 2.33 s, total: 36.9 s
Wall time: 34 s


### Create Claims Dataframe

The stemmed claims without stopwords are added to a claims dataframe here. In order to keep as much information as possible, each column below represent the claim at each level of the cleaning.

In [15]:
# zip together all the claim lists and create a dataframe
zipped_claims = list(zip(stemmed_claims, stemmed_sw_claims, tokenized_claims))
claims_ = pd.DataFrame(zipped_claims, columns = ['stemmed_claims', 'stemmed_stopword_claims', 'tokenized_claims'])

#display the claims dataframe
claims_.head()

,stemmed_claims,stemmed_stopword_claims,tokenized_claims
0,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow..."
1,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu..."
2,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,..."
3,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,..."
4,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,..."


### 1.3.2 Related Articles

Similar to the procedure performed on the claims, this section removes stopwords and stems the sentences extracted from the related articles.

In [16]:
%%time
# create a list of tokenized, non-stop words articles ~ takes 30 min
tokenized_articles = []
stemmed_art = []
stemmed_sw_art = []

for i in range(articles_cleaned.shape[0]):

    #--------------------------------------------------------------
    # stemming
    word_tokens = word_tokenize(articles_cleaned.Article[articles_cleaned.index[i]])
    stemmed_tok_art = []
    for w in word_tokens:
        stemmed_tok_art.append(ps.stem(w))
    stemmed_string = ' '.join(stemmed_tok_art)
    # stemmed_claims is a list of stemmed strings
    stemmed_art.append(stemmed_string)
    
    #--------------------------------------------------------------
    # remove stop words
    stemmed_sw_string = []
    word_tokens = word_tokenize(stemmed_art[i])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    stemmed_sw_string = ' '.join(filtered_sentence)
    # stemmed_sw_claims is a list of stemmed strings without stopwords
    stemmed_sw_art.append(stemmed_sw_string)    
    
    #--------------------------------------------------------------    
    # tokenize
    tokenized_ = word_tokenize(stemmed_sw_art[i])
    tokenized_articles.append(tokenized_)
    
    # print progress
    progress = round((i/articles_cleaned.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

progress: 100.0%
CPU times: user 31min 20s, sys: 21 s, total: 31min 41s
Wall time: 31min 24s


#### Sentences for Related Articles

In [17]:
%%time

# stem, remove stop words from sentences ~ takes 30 min

one_article = []
final_article_sentences = []
tokenized_sentences = []
final_article_tok_sentences = []

for i in range(article_sentences.shape[0]):
    # grab a series of sentence for one article
    sentence_list = article_sentences.Sentence[article_sentences.index[i]]
    for u in range(len(sentence_list)):
        # stem, stopwords for each sentence in an article
        sent = sentence_list[u]
        word_tokens = word_tokenize(sent)
        stemmed_tok_sent = []
        for w in word_tokens:
            stemmed_tok_sent.append(ps.stem(w))
        filtered_sentence = [w for w in stemmed_tok_sent if not w in stop_words]
        # append the tokenized strings
        tokenized_sentences.append(filtered_sentence)
        # append the strings
        stemmed_sw_string = ' '.join(filtered_sentence)
        one_article.append(stemmed_sw_string)
    # append each one_article to a full list of all articles
    final_article_sentences.append(one_article)  
    final_article_tok_sentences.append(tokenized_sentences)
    one_article = []
    tokenized_sentences = []
    
    # print progress
    progress = round((i/article_sentences.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")    

progress: 100.0%
CPU times: user 29min 27s, sys: 26.5 s, total: 29min 54s
Wall time: 29min 38s


The sentences from the related articles at each level of the cleaning are shown below.

In [18]:
# article_sentences is the dataframe that holds the sentences
article_sentences['cleaned_sentence'] = final_article_sentences
article_sentences['tokenized_cleaned_sentence'] = final_article_tok_sentences
article_sentences.head()

,Sentence,cleaned_sentence,tokenized_cleaned_sentence
125923,[fact check the five pillars of curriculum fac...,[fact check five pillar curriculum fact check ...,"[[fact, check, five, pillar, curriculum, fact,..."
57270,[the state department spent 52701 on the curta...,[state depart spent 52701 curtain un ambassado...,"[[state, depart, spent, 52701, curtain, un, am..."
144732,[us show of force sends russia a message in bl...,[us show forc send russia messag black sea was...,"[[us, show, forc, send, russia, messag, black,..."
28766,[mexico missing students protesters clash with...,[mexico miss student protest clash polic media...,"[[mexico, miss, student, protest, clash, polic..."
85305,[the us refugee resettlement program a primer ...,[us refuge resettl program primer policymak pr...,"[[us, refuge, resettl, program, primer, policy..."


### Create Related Articles DataFrame

The cleaned related articles as well as the sentences are combined together into a single dataframe with the code presented below.

In [19]:
# zip together all the articles and create a dataframe
zipped_articles = list(zip(stemmed_art, stemmed_sw_art, tokenized_articles, final_article_sentences, final_article_tok_sentences))
articles_ = pd.DataFrame(zipped_articles, columns = ['stemmed_articles', 'stemmed_stopword_articles', 'tokenized_articles', 'cleaned_sentence', 'tokenized_cleaned_sentence'])

# index the articles based on article ID
articles_.index = [articles_cleaned.index]

# display the articles_
articles_.head()

,stemmed_articles,stemmed_stopword_articles,tokenized_articles,cleaned_sentence,tokenized_cleaned_sentence
125923,fact check the five pillar of curriculum fact ...,fact check five pillar curriculum fact check s...,"[fact, check, five, pillar, curriculum, fact, ...",[fact check five pillar curriculum fact check ...,"[[fact, check, five, pillar, curriculum, fact,..."
57270,the state depart spent 52701 on the curtain in...,state depart spent 52701 curtain un ambassador...,"[state, depart, spent, 52701, curtain, un, amb...",[state depart spent 52701 curtain un ambassado...,"[[state, depart, spent, 52701, curtain, un, am..."
144732,us show of forc send russia a messag in black ...,us show forc send russia messag black sea wash...,"[us, show, forc, send, russia, messag, black, ...",[us show forc send russia messag black sea was...,"[[us, show, forc, send, russia, messag, black,..."
28766,mexico miss student protest clash with polic m...,mexico miss student protest clash polic media ...,"[mexico, miss, student, protest, clash, polic,...",[mexico miss student protest clash polic media...,"[[mexico, miss, student, protest, clash, polic..."
85305,the us refuge resettl program a primer for pol...,us refuge resettl program primer policymak pre...,"[us, refuge, resettl, program, primer, policym...",[us refuge resettl program primer policymak pr...,"[[us, refuge, resettl, program, primer, policy..."


### 1.4. Date Cleaning

For ease of data manipulation, the date is converted to "datetime" format and added as a new column "new_date".

In [20]:
# Convert date column to datetime format
data['new_date'] = pd.to_datetime(data['date'], dayfirst=True)
data.head()

,claim,claimant,date,id,label,related_articles,article_array,new_date
0,A line from George Orwell's novel 1984 predict...,,2017-07-17,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]",2017-07-17
1,Maine legislature candidate Leslie Gibson insu...,,2018-03-17,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]",2018-03-17
2,A 17-year-old girl named Alyssa Carson is bein...,,2018-07-18,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]",2018-07-18
3,In 1988 author Roald Dahl penned an open lette...,,2019-02-04,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]",2019-02-04
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22


### 1.5. Claimant Cleaning 

All empty claimant cells are replaced with "Unknown" in this section.

In [36]:
# fill missing claimants with "unknown"
data['claimant'] = data['claimant'].fillna('Unknown')
data['claimant'] = data['claimant'].replace('', 'Unknown')
data.head()

,claim,claimant,date,id,label,related_articles,article_array,new_date
0,A line from George Orwell's novel 1984 predict...,Unknown,2017-07-17,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]",2017-07-17
1,Maine legislature candidate Leslie Gibson insu...,Unknown,2018-03-17,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]",2018-03-17
2,A 17-year-old girl named Alyssa Carson is bein...,Unknown,2018-07-18,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]",2018-07-18
3,In 1988 author Roald Dahl penned an open lette...,Unknown,2019-02-04,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]",2019-02-04
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22


### 1.6. Finalize Train and Related Articles Data

With the train.json and related articles cleaned, this section forms two complete dataframes that hold the necessary information.

### 1.6.1 Finalize Cleaned Train Data

This section finalizes the cleaned train data which include the raw claim, cleaned claim, stemmed claims, stemmed claims without stopwords, tokenized claims, claimants, label, related article array and the dates.

In [45]:
# concatenate all work done above to a single frame
final_data = pd.concat([data.claim, cleaned_claim.claim, claims_.stemmed_claims, claims_.stemmed_stopword_claims, claims_.tokenized_claims, data.claimant, cleaned_claim.label, cleaned_claim.article_array, data.new_date], axis=1)

# rename columns for clarity
final_data.columns = ['raw_claim', 'cleaned_claim', 'stemmed_claims', 'stemmed_stopword_claims', 'tokenized_claim', 'claimant', 'label', 'article_array', 'new_date']

In [46]:
# this is equivalent to the "train.csv" that we were given, but cleaned
final_data.head()

,raw_claim,cleaned_claim,stemmed_claims,stemmed_stopword_claims,tokenized_claim,claimant,label,article_array,new_date
0,A line from George Orwell's novel 1984 predict...,a line from george orwells novel 1984 predicts...,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow...",Unknown,0,"[122094, 122580, 130685, 134765]",2017-07-17
1,Maine legislature candidate Leslie Gibson insu...,maine legislature candidate leslie gibson insu...,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu...",Unknown,2,"[106868, 127320, 128060]",2018-03-17
2,A 17-year-old girl named Alyssa Carson is bein...,a 17yearold girl named alyssa carson is being ...,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,...",Unknown,1,"[132130, 132132, 149722]",2018-07-18
3,In 1988 author Roald Dahl penned an open lette...,in 1988 author roald dahl penned an open lette...,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,...",Unknown,2,"[123254, 123418, 127464]",2019-02-04
4,"When it comes to fighting terrorism, ""Another ...",when it comes to fighting terrorism another th...,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,...",Hillary Clinton,2,"[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22


In [47]:
# # save to pkl
# final_data.to_pickle("./final_data.pkl")

### 1.6.2 Finalize Cleaned Related Articles Data

This part combines the necessary related article information in to a single dataframe. The columns contains the raw articles, cleaned articles, stemmed articles, stemmed articles without stopwords, tokenized articles, the cleaned sentences and lastly the tokenized cleaned sentences.

In [39]:
raw_article_list = articles.Article.tolist()
cleaned_article_list = articles_cleaned.Article.tolist()
final_articles_zipped = list(zip(raw_article_list, cleaned_article_list, stemmed_art, stemmed_sw_art, tokenized_articles, final_article_sentences, final_article_tok_sentences))
final_articles = pd.DataFrame(final_articles_zipped, columns = ['raw_articles', 'cleaned_articles', 'stemmed_articles', 'stemmed_stopwords_articles', 'tokenized_articles', 'cleaned_sentence', 'tokenized_cleaned_sentence'])
final_articles.index = [articles_cleaned.index]

In [40]:
final_articles.head()

,raw_articles,cleaned_articles,stemmed_articles,stemmed_stopwords_articles,tokenized_articles,cleaned_sentence,tokenized_cleaned_sentence
125923,FACT CHECK: The Five Pillars of Curriculum\nFA...,fact check the five pillars of curriculum fact...,fact check the five pillar of curriculum fact ...,fact check five pillar curriculum fact check s...,"[fact, check, five, pillar, curriculum, fact, ...",[fact check five pillar curriculum fact check ...,"[[fact, check, five, pillar, curriculum, fact,..."
57270,"The State Department Spent $52,701 on the Curt...",the state department spent 52701 on the curtai...,the state depart spent 52701 on the curtain in...,state depart spent 52701 curtain un ambassador...,"[state, depart, spent, 52701, curtain, un, amb...",[state depart spent 52701 curtain un ambassado...,"[[state, depart, spent, 52701, curtain, un, am..."
144732,US show of force sends Russia a message in Bla...,us show of force sends russia a message in bla...,us show of forc send russia a messag in black ...,us show forc send russia messag black sea wash...,"[us, show, forc, send, russia, messag, black, ...",[us show forc send russia messag black sea was...,"[[us, show, forc, send, russia, messag, black,..."
28766,Mexico missing students: Protesters clash with...,mexico missing students protesters clash with ...,mexico miss student protest clash with polic m...,mexico miss student protest clash polic media ...,"[mexico, miss, student, protest, clash, polic,...",[mexico miss student protest clash polic media...,"[[mexico, miss, student, protest, clash, polic..."
85305,The U.S. Refugee Resettlement Program: A Prime...,the us refugee resettlement program a primer f...,the us refuge resettl program a primer for pol...,us refuge resettl program primer policymak pre...,"[us, refuge, resettl, program, primer, policym...",[us refuge resettl program primer policymak pr...,"[[us, refuge, resettl, program, primer, policy..."


In [42]:
# final_articles.to_pickle('./final_articles.pkl')

# CHECKPOINT

**At this point, there are 2 dataframes as shown above: final_data and final_articles**

## 2. Exploratory Data Analysis

<-- insert brief description of the purpose of this section -->

### 2.1 Date Features 

This section is focused on creating features from the date information provided.

<-- include a list of features added and their description -->

In [ ]:
# final_data = pd.read_pickle('./final_data.pkl')

In [49]:
# create new feature with consecutive days since January 1st, 1986
final_data['start_date'] = pd.to_datetime('1986-01-01', format='%Y-%m-%d')
final_data['cont_days'] = (final_data['new_date'] - final_data['start_date']).dt.days
final_data = final_data.drop(['start_date'], axis=1)

# Convert Year and Month features in to int (instead of str before), can be kept as int since it is ordinal

#Year
final_data['Year'] = final_data['new_date'].apply(lambda x: "%d" % (x.year))
final_data['Year'] = final_data['Year'].astype(int)

# Month
final_data['Month'] = final_data['new_date'].apply(lambda x: "%d" % (x.month))
final_data['Month'] = final_data['Month'].astype(int)

In [50]:
final_data.head()

,raw_claim,cleaned_claim,stemmed_claims,stemmed_stopword_claims,tokenized_claim,claimant,label,article_array,new_date,cont_days,Year,Month
0,A line from George Orwell's novel 1984 predict...,a line from george orwells novel 1984 predicts...,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow...",Unknown,0,"[122094, 122580, 130685, 134765]",2017-07-17,11520,2017,7
1,Maine legislature candidate Leslie Gibson insu...,maine legislature candidate leslie gibson insu...,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu...",Unknown,2,"[106868, 127320, 128060]",2018-03-17,11763,2018,3
2,A 17-year-old girl named Alyssa Carson is bein...,a 17yearold girl named alyssa carson is being ...,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,...",Unknown,1,"[132130, 132132, 149722]",2018-07-18,11886,2018,7
3,In 1988 author Roald Dahl penned an open lette...,in 1988 author roald dahl penned an open lette...,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,...",Unknown,2,"[123254, 123418, 127464]",2019-02-04,12087,2019,2
4,"When it comes to fighting terrorism, ""Another ...",when it comes to fighting terrorism another th...,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,...",Hillary Clinton,2,"[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22,11038,2016,3


In [51]:
# final_data.to_pickle('./final_data_dates.pkl')

<-- Jacob's code should be inserted here working off of final_data_dates.pkl -->

### 2.2 Claimant Features 

<-- Jacob's code should be inserted here working off of final_Dat_dates.pkl ==>

### 2.3 Doc2Vec Features

In this section, Doc2Vec is implemented to generate claim and related article sentence vectors. Cosine similarity is used to find the top 5 related sentences from each related article per claim, and those are added as features.

In [ ]:
# # read the data
# data = pd.read_pickle("./final_data_dates.pkl")
# articles = pd.read_pickle('./final_articles.pkl')
data = final_data
articles = final_articles

### 2.3.1 Preprocess the Data 

This section preprocesses the claims and related articles data to prepare for Doc2Vec training. To start, a label is generated for every sentence of every related article. For example "125923-5" represents the 5th sentence of article ID 125923.

In [55]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy import spatial
import random

In [56]:
%%time

# 2 minutes to run
# create a label for every sentence based on article ID (eg. 12345-1, 12345-2, 12345-3 ... etc.)

full_sentences_ID = []
for i in range(articles.shape[0]):
    sentence_ID_list = []
    sentence_number = 0
    sentences = articles.tokenized_cleaned_sentence.loc[articles.index[i]]
    for u in range(len(sentences)):
        sentence_ID = str(articles.index[i][0]) +  '-' + str(sentence_number)
        sentence_number += 1
        sentence_ID_list.append(sentence_ID)
    full_sentences_ID.append(sentence_ID_list)
    
    # print progress
    progress = round((i/articles.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [58]:
# combine the tokenized sentences with their ID
articles_tok_sent = articles.tokenized_cleaned_sentence.to_frame()
articles_tok_sent['sentence_ID'] = full_sentences_ID
articles_tok_sent.head()

,tokenized_cleaned_sentence,sentence_ID
125923,"[[fact, check, five, pillar, curriculum, fact,...","[125923-0, 125923-1, 125923-2, 125923-3, 12592..."
57270,"[[state, depart, spent, 52701, curtain, un, am...","[57270-0, 57270-1, 57270-2, 57270-3, 57270-4, ..."
144732,"[[us, show, forc, send, russia, messag, black,...","[144732-0, 144732-1, 144732-2, 144732-3, 14473..."
28766,"[[mexico, miss, student, protest, clash, polic...","[28766-0, 28766-1, 28766-2, 28766-3, 28766-4, ..."
85305,"[[us, refuge, resettl, program, primer, policy...","[85305-0, 85305-1, 85305-2, 85305-3, 85305-4, ..."


In [ ]:
# %%time

# # combine the sentence list and index list ~ takes 1.5 hours to run

# article_sent_list = []
# article_sent_ID = []

# for i in range(articles_tok_sent.shape[0]):
#     one_article_sent = articles_tok_sent.tokenized_cleaned_sentence.loc[articles.index[i]]
#     article_sent_list = article_sent_list + one_article_sent
#     one_article_ID = articles_tok_sent.sentence_ID.loc[articles.index[i]]
#     article_sent_ID = article_sent_ID + one_article_ID
    
#     # print progress
#     progress = round((i/articles_tok_sent.shape[0])*100,2)
#     clear_output(wait=True)
#     print("progress: " + str(progress) + "%")

In [ ]:
# # create a data frame of the independent sentences and ID

# # create copies
# l1 = article_sent_list
# l2 = article_sent_ID
# # make dataframe
# article_information = pd.Series(l2).to_frame()
# article_information.columns = ['sentence_ID']
# article_information['sentences'] = l1
# # display it
# article_information.head()

In [ ]:
# # save it to pickle
# article_information.to_pickle("./article_sentences_ind.pkl")

Every sentence of every related article along with the respective ID's are added to the following dataframe for Doc2Vec training.

In [62]:
sentences_pkl = pd.read_pickle("./article_sentences_ind.pkl")
# sentences_pkl = article_information
sentences_pkl.head()

,sentence_ID,sentences
0,125385-0,"[dr, ben, carson, welfar, benefactor]"
1,125385-1,"[initi, unlik, clearli, formid, contend, 2016,..."
2,125385-2,"[unlik, chri, christi, rand, paul, mike, hucka..."
3,125385-3,"[carson, becam, somewhat, overnight, sensat, f..."
4,125385-4,"[earliest, version, meme, date, least, decemb,..."


In [63]:
# extract the lists from the dataframes
sentence_ID_list = sentences_pkl.sentence_ID.tolist()
sentence_list = sentences_pkl.sentences.tolist()

### 2.3.2 Prepare for Training

The related article sentences along with the claims are added together with their ID's in this section for Doc2Vec training.

In [66]:
%%time
    
# create a list of claim index ranging from 0 - 15554
claim_range = range(0, 15555)
claim_index = []
for i in claim_range:
    claim_index.append(i)

# create a list of lists of data.tokenized_claim
tokenized_claims = []
for i in range(data.shape[0]):
    tokenized_claims.append(data.tokenized_claim[i])
    
# convert claim index to string to match the sentences ID
claim_str_index = list(map(str, claim_index))

# form full list for training
full_text_list = sentence_list + tokenized_claims
full_ID_list = sentence_ID_list + claim_str_index

CPU times: user 450 ms, sys: 4 µs, total: 450 ms
Wall time: 449 ms


In [67]:
# combine above 2 lists in to a dictionary for debugging
complete_data = dict(zip(full_ID_list, full_text_list))

### 2.3.3 Doc2Vec Training 

The code to enable Doc2Vec training is implemented below. Some key parameters are a vector size of 150 and 30 epochs.

In [ ]:
# %%time

# #create tagged data to train doc2vec w claim
# tagged_data = [TaggedDocument(words=full_text_list[i], tags=[full_ID_list[i]]) for i in range(len(full_text_list))]

In [ ]:
# %%time

# # 6 hours to run

# import multiprocessing
# n_cpu = multiprocessing.cpu_count()

# # setup training
# vec_size = 150
# model = Doc2Vec(dm = 1, vector_size = vec_size, min_count = 2, workers = n_cpu, epochs=30)

# # build vocab
# model.build_vocab(tagged_data)

# # train
# model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# # save model
# model.save("30epoch_150vec.model")
# print("Model Saved")

### 2.3.4 Find Similarities 

Load the trained model.

In [68]:
%%time

# load the model
model = Doc2Vec.load("30epoch_150vec.model")

CPU times: user 14 s, sys: 2.51 s, total: 16.5 s
Wall time: 19.9 s


### 2.3.4.1 Finding Top 5 Sentences per Related Articles per Claim 

Find the top 5 sentences for each related article per claim. For example, Claim 0 has 5 related articles. Within each related article, top 5 most similar sentences are extracted. As a result, 25 total sentences are extracted from the 5 related articles. The similarity score of the sentences per related article is averaged, and then the 5 averages are averaged again to create an average similarity score between the claim and the 5 top 5 sentences of each 5 related articles.

In [71]:
%%time

# ~ 6 minutes to run
# find the top 5 sentence ID from each related article with the highest similarity score with the claim

final_best_sentences_ID = []
for i in range(data.shape[0]): # iterating over every claim
    one_claim_array = data.article_array.loc[i]
    best_sentences_one_claim = []
    for u in range(len(one_claim_array)): # iterating every article of one claim
        one_article_ID = one_claim_array[u]
        sent_list = articles.tokenized_cleaned_sentence.loc[str(one_article_ID)].iloc[0]
        one_article_sim_list=[]
        for y in range(len(sent_list)): #iterating over every sentence of one article
            v1 = model.docvecs[str(i)]
            sentence_number = str(str(one_article_ID) + '-' + str(y))
            v2 = model.docvecs[sentence_number]
            similarity = 1 - spatial.distance.cosine(v1, v2)
            one_article_sim_list.append(similarity)
#             print("ID: " + str(i) + " " + str(result))
        a = numpy.array(one_article_sim_list)
        best_sentences_one_article = heapq.nlargest(5, range(len(a)), a.take)   
        best_sentences_one_claim.append(best_sentences_one_article)
    final_best_sentences_ID.append(best_sentences_one_claim)
    # print progress
    progress = round((i/data.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

# add it to the data frame
data['best_sentences_ID'] = final_best_sentences_ID

progress: 99.99%
CPU times: user 6min 1s, sys: 7.29 s, total: 6min 8s
Wall time: 6min 2s


In [72]:
%%time

# ~ 2 minuts to run
# calculate the top 5 similarity scores

end = data.shape[0]
# end = 2
avg_sim_for_one_claim_list = []
avg_sim_one_claim = []
full_sentence_ID = []
for i in range(end): # per row
    one_claim_articles_ID = data.article_array.loc[i]
    one_claim_sentences_ID = data.best_sentences_ID.loc[i]
    avg_sim_for_one_article = []
    article_sentence_ID = []
#     print("claim: " + str(i))
    for u in range(len(one_claim_articles_ID)): # article_array index
        sim_for_one_article = []
        sentence_ID_list = []
        for y in range(len(one_claim_sentences_ID[u])): # sentence ID index      
            sentence_ID = str(one_claim_articles_ID[u]) + '-' + str(data.best_sentences_ID.loc[i][u][y])
            sentence_ID_list.append(sentence_ID)
            # calculate sim score
            v1 = model.docvecs[str(i)]
            v2 = model.docvecs[sentence_ID]
            similarity = 1 - spatial.distance.cosine(v1, v2)
            # similarity between claim and each sentence is calculated
            sim_for_one_article.append(similarity)
#             print(str(sentence_ID) + ": " + str(similarity))
        # the similarity between claim and each sentence per article is averaged
        avg_sim = sum(sim_for_one_article)/len(sim_for_one_article)
        avg_sim_for_one_article.append(avg_sim)
#         print("")
        article_sentence_ID.append(sentence_ID_list)
    
    # create a list of sentence ID's
    full_sentence_ID.append(article_sentence_ID)
    
    #calculate average scores for each claim
    avg_sim_ = sum(avg_sim_for_one_article)/len(avg_sim_for_one_article)
    avg_sim_one_claim.append(avg_sim_)
    
    # a list of lists, big list per claim, and small list per article
    avg_sim_for_one_claim_list.append(avg_sim_for_one_article)
    
    # print progress
    progress = round((i/end)*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

progress: 99.99%
CPU times: user 1min 9s, sys: 2.65 s, total: 1min 11s
Wall time: 1min 9s


In [74]:
# add to data frame
data['full_sentence_ID'] = full_sentence_ID
data['avg_sentence_sim'] = avg_sim_for_one_claim_list
data['avg_sim_score'] = avg_sim_one_claim

In [75]:
data.head()

,raw_claim,cleaned_claim,stemmed_claims,stemmed_stopword_claims,tokenized_claim,claimant,label,article_array,new_date,cont_days,Year,Month,best_sentences_ID,full_sentence_ID,avg_sentence_sim,avg_sim_score
0,A line from George Orwell's novel 1984 predict...,a line from george orwells novel 1984 predicts...,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow...",Unknown,0,"[122094, 122580, 130685, 134765]",2017-07-17,11520,2017,7,"[[0, 21, 15, 20, 10], [11, 24, 12, 13, 26], [3...","[[122094-0, 122094-21, 122094-15, 122094-20, 1...","[0.26672267615795137, 0.24059542417526245, 0.1...",0.185311
1,Maine legislature candidate Leslie Gibson insu...,maine legislature candidate leslie gibson insu...,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu...",Unknown,2,"[106868, 127320, 128060]",2018-03-17,11763,2018,3,"[[11, 7, 13, 3, 4], [22, 24, 0, 1, 19], [0, 8,...","[[106868-11, 106868-7, 106868-13, 106868-3, 10...","[0.382437926530838, 0.374161022901535, 0.28886...",0.348489
2,A 17-year-old girl named Alyssa Carson is bein...,a 17yearold girl named alyssa carson is being ...,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,...",Unknown,1,"[132130, 132132, 149722]",2018-07-18,11886,2018,7,"[[22, 30, 8, 32, 29], [3, 6, 59, 60, 83], [6, ...","[[132130-22, 132130-30, 132130-8, 132130-32, 1...","[0.45887559056282046, 0.6421806216239929, 0.29...",0.464089
3,In 1988 author Roald Dahl penned an open lette...,in 1988 author roald dahl penned an open lette...,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,...",Unknown,2,"[123254, 123418, 127464]",2019-02-04,12087,2019,2,"[[51, 41, 70, 103, 30], [107, 102, 61, 191, 98...","[[123254-51, 123254-41, 123254-70, 123254-103,...","[0.7696409344673156, 0.7453237056732178, 0.755...",0.756903
4,"When it comes to fighting terrorism, ""Another ...",when it comes to fighting terrorism another th...,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,...",Hillary Clinton,2,"[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22,11038,2016,3,"[[168, 170, 45, 191, 32], [10, 12, 4, 16, 25],...","[[41099-168, 41099-170, 41099-45, 41099-191, 4...","[0.4999494135379791, 0.408842134475708, 0.3396...",0.411560


### 2.3.4.2 Finding Top 5 Sentences Amongst all Related Articles per Claim

In this section, instead of finding top 5 sentences from each related article, the top 5 sentences from ALL related articles of a claim is extracted. The average similarity score between the claim and the top 5 sentences are calculated and added as a feature.

In [76]:
%%time

# 1 min to run
# finding top 5 sentences amongst all relateld articles per claim

end = data.shape[0]
# end = 10

sim_score_dict = {}

final_top_sentences = []

for i in range(end):
    top_sentence_list = data.full_sentence_ID.loc[i]
    sim_score_tuple = []
    for u in range(len(top_sentence_list)):
        for y in range(len(top_sentence_list[u])):
            sentence_ID = top_sentence_list[u][y]
            # calculate sim score
            v1 = model.docvecs[str(i)]
            v2 = model.docvecs[sentence_ID]
            similarity = 1 - spatial.distance.cosine(v1, v2)
#             sim_for_one_claim.append(similarity)
            sim_score_tuple.append(tuple((sentence_ID, similarity)))
#             print(str(sentence_ID) + ": " + str(similarity))
    a = nlargest(5, sim_score_tuple, key=itemgetter(1))
    top_sentences = [q[0] for q in a]
    # list of lists: final top 5 sentences amongst all related articles per claim
    final_top_sentences.append(top_sentences)
    
    # print progress
    progress = round((i/end)*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

# save it to the dataframe
data['top_5_sentences'] = final_top_sentences

progress: 99.99%
CPU times: user 50.7 s, sys: 3.26 s, total: 54 s
Wall time: 51 s


In [77]:
%%time

# average similarity scores of top 5 sentences amongst all relateld articles per claim

end = data.shape[0]
# end = 10

total_avg_sim = []

for i in range(end):
    top_sentence_list = data.top_5_sentences.loc[i]
    top_sim_scores = []
    for u in range(len(top_sentence_list)):
        sentence_ID = top_sentence_list[u]
        # calculate sim score
        v1 = model.docvecs[str(i)]
        v2 = model.docvecs[sentence_ID]
        similarity = 1 - spatial.distance.cosine(v1, v2)       
        top_sim_scores.append(similarity)
        
    avg_sim = sum(top_sim_scores)/len(top_sim_scores)
    total_avg_sim.append(avg_sim)
    
    # print progress
    progress = round((i/end)*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

# save it to the dataframe
data['top5_avg_sim'] = total_avg_sim

progress: 95.2%


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [78]:
data.head()

,raw_claim,cleaned_claim,stemmed_claims,stemmed_stopword_claims,tokenized_claim,claimant,label,article_array,new_date,cont_days,Year,Month,best_sentences_ID,full_sentence_ID,avg_sentence_sim,avg_sim_score,top_5_sentences,top5_avg_sim
0,A line from George Orwell's novel 1984 predict...,a line from george orwells novel 1984 predicts...,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow...",Unknown,0,"[122094, 122580, 130685, 134765]",2017-07-17,11520,2017,7,"[[0, 21, 15, 20, 10], [11, 24, 12, 13, 26], [3...","[[122094-0, 122094-21, 122094-15, 122094-20, 1...","[0.26672267615795137, 0.24059542417526245, 0.1...",0.185311,"[122094-0, 122580-11, 122094-21, 122094-15, 12...",0.291527
1,Maine legislature candidate Leslie Gibson insu...,maine legislature candidate leslie gibson insu...,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu...",Unknown,2,"[106868, 127320, 128060]",2018-03-17,11763,2018,3,"[[11, 7, 13, 3, 4], [22, 24, 0, 1, 19], [0, 8,...","[[106868-11, 106868-7, 106868-13, 106868-3, 10...","[0.382437926530838, 0.374161022901535, 0.28886...",0.348489,"[106868-11, 106868-7, 127320-22, 127320-24, 12...",0.400307
2,A 17-year-old girl named Alyssa Carson is bein...,a 17yearold girl named alyssa carson is being ...,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,...",Unknown,1,"[132130, 132132, 149722]",2018-07-18,11886,2018,7,"[[22, 30, 8, 32, 29], [3, 6, 59, 60, 83], [6, ...","[[132130-22, 132130-30, 132130-8, 132130-32, 1...","[0.45887559056282046, 0.6421806216239929, 0.29...",0.464089,"[132132-3, 132132-6, 132132-59, 132132-60, 132...",0.642181
3,In 1988 author Roald Dahl penned an open lette...,in 1988 author roald dahl penned an open lette...,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,...",Unknown,2,"[123254, 123418, 127464]",2019-02-04,12087,2019,2,"[[51, 41, 70, 103, 30], [107, 102, 61, 191, 98...","[[123254-51, 123254-41, 123254-70, 123254-103,...","[0.7696409344673156, 0.7453237056732178, 0.755...",0.756903,"[127464-222, 123254-51, 123254-41, 123254-70, ...",0.775002
4,"When it comes to fighting terrorism, ""Another ...",when it comes to fighting terrorism another th...,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,...",Hillary Clinton,2,"[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22,11038,2016,3,"[[168, 170, 45, 191, 32], [10, 12, 4, 16, 25],...","[[41099-168, 41099-170, 41099-45, 41099-191, 4...","[0.4999494135379791, 0.408842134475708, 0.3396...",0.411560,"[41099-168, 95344-259, 95344-247, 95344-74, 95...",0.547029


In [ ]:
# # save to data frame
# data.to_pickle("./data_sentence_simscore.pkl")

### 2.3.5 Feature Encoding

In this section, the claims, related article sentences and similarity scores are encoded in to numerical values for our fake news classfication model.

In [79]:
# # load the files

# full_data = pd.read_pickle("./data_sentence_simscore.pkl")
full_data = data

### 2.3.5.1 Feature Encode the Claims

This section applies the trained Doc2Vec model to generate a vector of size 150 for each claim.

In [80]:
# create a list of lists containing the claim's vector
claim_vectors = []
for i in range(0, 15555):
    claim_vectors.append(model.docvecs[str(i)])

# create a list of column names
column_name_list = []
for i in range(0,150):
    column_name = "claim_vec_" + str(i)
    column_name_list.append(column_name)

# turn the claim vectors in to a dataframe and rename the columns accordingly
claim_features = pd.DataFrame.from_records(claim_vectors)
claim_features.columns = column_name_list

In [81]:
claim_features.head()

,claim_vec_0,claim_vec_1,claim_vec_2,claim_vec_3,claim_vec_4,claim_vec_5,claim_vec_6,claim_vec_7,claim_vec_8,claim_vec_9,...,claim_vec_140,claim_vec_141,claim_vec_142,claim_vec_143,claim_vec_144,claim_vec_145,claim_vec_146,claim_vec_147,claim_vec_148,claim_vec_149
0,-0.212091,0.127670,0.106502,0.391641,0.303139,-0.248252,0.236494,0.104728,-0.271278,0.377465,...,-0.283741,-0.166108,-0.125862,0.316487,-0.086704,-0.015346,0.263394,0.236137,0.191386,-0.144861
1,0.512708,0.203220,0.151913,-0.307748,-0.426895,0.143223,0.293944,-0.129267,0.512433,0.322509,...,-0.216366,0.181766,0.237602,0.279045,-0.126447,-0.209171,-0.191039,0.158190,-0.066281,-0.247980
2,0.145354,0.215115,0.078994,0.009343,0.213732,-0.022070,0.126207,-0.130475,-0.093990,0.159534,...,-0.155529,-0.015081,0.274674,0.272769,-0.237696,0.130785,-0.032134,0.104650,0.257081,0.016224
3,-0.138208,0.415532,-0.018799,0.094876,0.292537,-0.157601,0.552872,-0.179123,0.080213,-0.131668,...,-0.336580,-0.263726,-0.080669,0.152311,-0.093220,0.300876,-0.068453,0.307259,0.150425,-0.206309
4,0.126634,0.311318,-0.125836,-0.143917,-0.092023,-0.118011,-0.178362,-0.358138,0.349950,-0.148357,...,-0.029561,-0.171704,0.290350,0.440742,-0.249992,0.430053,-0.086707,0.252833,-0.295727,0.226280


### 2.3.5.2 Feature Encode the Top 5 Related Article Sentences for Each Claim

In thi section, the top 5 sentences from all related articles per claim are encoded by using the Doc2Vec model to generate a vector for each sentence. (5 sentences, 150 vectors each, 750 columns of features in total).

In [82]:
%%time

end = full_data.shape[0]
# end = 1
full_sentence_vector_list = []
for i in range(end):
    top_5_sentences = full_data.top_5_sentences.loc[i]
    sentence_vector_list = []
    for u in range(len(top_5_sentences)):
        sentence_ID = full_data.top_5_sentences.loc[i][u]
        sentence_vector = model.docvecs[sentence_ID]
        sentence_vector_list.extend(sentence_vector)
    full_sentence_vector_list.append(sentence_vector_list)

    # print progress
    progress = round((i/end)*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

# create column names
complete_column_name=[]
for i in range(1, 6):
    column_name_list = []
    for u in range(0,150):
        column_name = "sent_" + str(i) + "vec_" + str(u)
        column_name_list.append(column_name)
    complete_column_name.extend(column_name_list)

# convert full sentence features in to dataframe and name the columns accordingly
sentence_features = pd.DataFrame.from_records(full_sentence_vector_list)
sentence_features.columns = complete_column_name

progress: 99.99%
CPU times: user 24.3 s, sys: 2.6 s, total: 26.9 s
Wall time: 23.9 s


In [83]:
sentence_features.head()

,sent_1vec_0,sent_1vec_1,sent_1vec_2,sent_1vec_3,sent_1vec_4,sent_1vec_5,sent_1vec_6,sent_1vec_7,sent_1vec_8,sent_1vec_9,...,sent_5vec_140,sent_5vec_141,sent_5vec_142,sent_5vec_143,sent_5vec_144,sent_5vec_145,sent_5vec_146,sent_5vec_147,sent_5vec_148,sent_5vec_149
0,0.010075,-0.034665,0.158326,0.114454,0.167062,-0.325514,0.343727,-0.181544,0.427879,-0.064174,...,-0.329464,-0.123360,-0.106719,0.596236,-0.361782,-0.154024,0.078335,0.041940,-0.059460,0.239629
1,0.537097,0.346272,-0.127193,0.008937,-0.498761,0.209842,-0.136562,-0.121760,0.598921,0.272825,...,0.310716,1.026762,1.642795,0.402427,-0.656952,-0.680550,1.451327,0.068513,0.286796,0.028587
2,-0.034549,0.159652,0.068537,-0.108595,0.401951,0.025913,0.490867,-0.182542,0.126992,0.227730,...,-0.187836,-0.225113,0.046499,0.118527,0.053234,0.126584,-0.112338,0.300140,0.125310,0.023155
3,-0.006104,0.218761,-0.044763,-0.014197,0.140955,0.030794,0.227532,-0.064394,0.077120,0.073692,...,-0.184586,-0.138933,0.027737,0.214644,-0.101030,0.155582,0.019132,0.178928,0.125431,0.017242
4,0.143222,0.262105,-0.407402,-0.173036,-0.196569,0.068167,0.154569,-0.175273,0.005277,0.168737,...,-0.060368,-0.066583,0.038418,0.146584,-0.051212,0.113704,-0.006178,0.114713,0.056479,0.011474


### 2.3.5.3 Feature Encode the Similarity Scores

The average similarity scores are appended here.

In [84]:
sim_score_features = pd.concat([full_data.avg_sim_score, full_data.top5_avg_sim], axis=1)
sim_score_features.columns = ["avg_sent_sim", "avg_5_sent_sim"]

### 2.3.6 Combine All Features (Claim Vector, Top 5 Related Sentence Vector, Similarity Scores)

This section combines all features mentioned above in to one complete dataframe.

In [85]:
final_similiarity_sentences_features = pd.concat([claim_features, sentence_features, sim_score_features], axis=1)

In [87]:
# replace nan values with zero
final_similiarity_sentences_features.fillna(0, inplace = True)

In [89]:
# # save the final features to a pickle
# final_similiarity_sentences_features.to_pickle("./doc2vec_features.pkl")

In [90]:
final_similiarity_sentences_features.head()

,claim_vec_0,claim_vec_1,claim_vec_2,claim_vec_3,claim_vec_4,claim_vec_5,claim_vec_6,claim_vec_7,claim_vec_8,claim_vec_9,...,sent_5vec_142,sent_5vec_143,sent_5vec_144,sent_5vec_145,sent_5vec_146,sent_5vec_147,sent_5vec_148,sent_5vec_149,avg_sent_sim,avg_5_sent_sim
0,-0.212091,0.127670,0.106502,0.391641,0.303139,-0.248252,0.236494,0.104728,-0.271278,0.377465,...,-0.106719,0.596236,-0.361782,-0.154024,0.078335,0.041940,-0.059460,0.239629,0.185311,0.291527
1,0.512708,0.203220,0.151913,-0.307748,-0.426895,0.143223,0.293944,-0.129267,0.512433,0.322509,...,1.642795,0.402427,-0.656952,-0.680550,1.451327,0.068513,0.286796,0.028587,0.348489,0.400307
2,0.145354,0.215115,0.078994,0.009343,0.213732,-0.022070,0.126207,-0.130475,-0.093990,0.159534,...,0.046499,0.118527,0.053234,0.126584,-0.112338,0.300140,0.125310,0.023155,0.464089,0.642181
3,-0.138208,0.415532,-0.018799,0.094876,0.292537,-0.157601,0.552872,-0.179123,0.080213,-0.131668,...,0.027737,0.214644,-0.101030,0.155582,0.019132,0.178928,0.125431,0.017242,0.756903,0.775002
4,0.126634,0.311318,-0.125836,-0.143917,-0.092023,-0.118011,-0.178362,-0.358138,0.349950,-0.148357,...,0.038418,0.146584,-0.051212,0.113704,-0.006178,0.114713,0.056479,0.011474,0.411560,0.547029


## 2.4 Sentiment Features 

## 2.5 Exploration of Features 

## 3. Feature Selection

## 4. Model Implementation 

## 5. Model Tuning 

## 6. Results 